<a href="https://colab.research.google.com/github/ballmdr/Forex-Prediction-Machine_Learning/blob/master/pyTorch_RNN_LSTM_GRU_(test_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [0]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

In [0]:

def createInputs(text):
    inputs = []
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
        #print(v.reshape(1,-1))
        #print('-----')

    return inputs

In [0]:
vocab = set()
for text in train_data.keys():
    for w in text.split(' '):
        vocab.add(w)
        
vocab = list(vocab)
vocab_size = len(vocab)

word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = {i: w for i, w in enumerate(vocab) }

x_train = []
y_train = []

for key, val in train_data.items():
    x_train.append(key)
    y_train.append(int(val))
    
#x_test = np.array(x_test, np.float32)
#y_train = np.array(y_train, np.float32)

x_test = []
y_test = []

for key, val in test_data.items():
    x_test.append(key)
    y_test.append(int(val))
    
#x_test = np.array(x_test, np.float32)
#y_test = np.array(y_test, np.float32)

In [17]:
x_train

['good',
 'bad',
 'happy',
 'sad',
 'not good',
 'not bad',
 'not happy',
 'not sad',
 'very good',
 'very bad',
 'very happy',
 'very sad',
 'i am happy',
 'this is good',
 'i am bad',
 'this is bad',
 'i am sad',
 'this is sad',
 'i am not happy',
 'this is not good',
 'i am not bad',
 'this is not sad',
 'i am very happy',
 'this is very good',
 'i am very bad',
 'this is very sad',
 'this is very happy',
 'i am good not bad',
 'this is good not bad',
 'i am bad not good',
 'i am good and happy',
 'this is not good and not happy',
 'i am not at all good',
 'i am not at all bad',
 'i am not at all happy',
 'this is not at all sad',
 'this is not at all happy',
 'i am good right now',
 'i am bad right now',
 'this is bad right now',
 'i am sad right now',
 'i was good earlier',
 'i was happy earlier',
 'i was bad earlier',
 'i was sad earlier',
 'i am very bad right now',
 'this is very good right now',
 'this is very sad right now',
 'this was bad earlier',
 'this was very good earli

In [0]:
class RNN(nn.Module):
    def __init__(self, hidden, K, vocab_size):
        super(RNN, self).__init__()
        
        self.linearH = nn.Linear(vocab_size + hidden, hidden)
        self.linearO = nn.Linear(vocab_size + hidden, K)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        x = torch.cat((x, h), 1)
        h = torch.tanh(self.linearH(x))
        out = torch.sigmoid(self.linearO(x))
        
        return out, h
      
      
class LSTM(nn.Module):
  def __init__(self, hidden, output_size, vocab_size):
    super(LSTM, self).__init__()
    
    self.linearFt = nn.Linear(vocab_size + hidden, hidden)
    self.linearIt = nn.Linear(vocab_size + hidden, hidden)
    self.linearCt1 = nn.Linear(vocab_size + hidden, hidden)
    self.linearOt = nn.Linear(vocab_size + hidden, hidden)
    self.linearO = nn.Linear(vocab_size + hidden, output_size)
    
  def forward(self, x, h, c):
    combined = torch.cat((x, h), 1)
    ft = torch.sigmoid(self.linearFt(combined))
    it = torch.sigmoid(self.linearIt(combined))
    ct1 = torch.tanh(self.linearCt1(combined))
    c = (ft*c) + (it * ct1)
    
    ot = torch.sigmoid(self.linearOt(combined))
    h = ot * torch.tanh(c)
    
    out = torch.sigmoid(self.linearO(combined))
    
    return out, h, c

class GRU(nn.Module):
  
  def __init__(self, hidden, output_size, vocab_size):
    super(GRU, self).__init__()
    
    self.linearZt = nn.Linear(vocab_size + hidden, hidden)
    self.linearRt = nn.Linear(vocab_size + hidden, hidden)
    self.linearHt = nn.Linear(vocab_size + hidden, hidden)
    
    self.linearO = nn.Linear(vocab_size + hidden, output_size)
    
  def forward(self, x, h):
    combined = torch.cat((x, h), 1)
    zt = torch.sigmoid(self.linearZt(combined))
    rt = torch.sigmoid(self.linearRt(combined))
    combined2 = torch.cat((x, rt*h), 1)
    ht = torch.tanh(self.linearHt(combined2))
    h = (1-zt) * h + zt * ht
    
    out = torch.sigmoid(self.linearO(combined))
  
    return out, h
  
  
hidden = 3
K = 1
vocab_size = len(vocab)
learning_rate = 0.5
epochs = 5000
train_loss = []
test_loss = []

#model = RNN(hidden, K, vocab_size)
model = GRU(hidden, K, vocab_size)
model.cuda()
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000, 2000, 3000, 4000], gamma=0.1)


for e in range(epochs):
    model.train()
    running_loss = 0.0
    testing_loss = 0.0
    
    for x, y in zip(x_train, y_train):
      
        h = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        c = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        y_tensor = torch.tensor(y, dtype=torch.float32, device='cuda')
        inputs = createInputs(x)
        
        optimizer.zero_grad()
        
        for w in inputs:
            x_tensor = torch.tensor(w, dtype=torch.float32, device='cuda')
            x_tensor = x_tensor.view(1,-1)
            out, h = model(x_tensor, h)
            #index = np.where(w==1)[0][0]
            #print(idx_to_word[index])

        loss = loss_fn(out, y_tensor)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    run_loss = running_loss/len(x_train)
    train_loss.append(run_loss)
    scheduler.step()

    model.eval()
    test_count = 5
    for j in range(test_count):
        index = np.random.randint(0, len(test_data)-1)
        h = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        c = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        line = createInputs(x_test[index])
        for w in line:
            w = torch.tensor(w.reshape(1,-1), dtype=torch.float32, device='cuda')
            out, h = model(w, h)

        loss = loss_fn(out, torch.tensor(int(y_test[index]), dtype=torch.float32).view(1,1).to('cuda'))
        testing_loss += loss.item()
    
    avg_loss = testing_loss / test_count
    test_loss.append(avg_loss)
    
    if e % 100 == 0:
        print(e, ' Train Loss: ', run_loss, ' Test Loss: ', avg_loss)
    
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


0  Train Loss:  0.8455213755369186  Test Loss:  0.8408667087554932
100  Train Loss:  0.37762670649079094  Test Loss:  0.28701190433930607
200  Train Loss:  0.373026242161547  Test Loss:  0.996937096118927
300  Train Loss:  0.3709644578369174  Test Loss:  0.36741158890072256
400  Train Loss:  0.3697484662753313  Test Loss:  0.6553669616580009
500  Train Loss:  0.36892575475062117  Test Loss:  0.47544659962877633
600  Train Loss:  0.3683218296997626  Test Loss:  0.25219378031324596
700  Train Loss:  0.3678538532965768  Test Loss:  0.5058298716787248
